# Lazy Services
PyJudo lets you define dependencies as `Lazy`. This means that the resolution and instanciation of the service (if transient and/or uninitialised) is deferred until the service is accessed.
This allows for finer control over the creation of dependencies, including lazily passing keyword arguments. For example, if you had an `Manager` which creates a `Service`, which requires a `service_input` that is unknown on the creation of the `Manager`:

```python
class Manager(IManager):
    def create_service(service_input) -> Service:
        return Service(service_input)
```

You could give the `Manager` the `ServiceContainer` and manually get a new instance of `Service` from it, passing a the `service_input`:

```python
class Manager(IManager):
    def __init__(self, services: IServiceContainer):
        self._services = services
    
    def create_service(service_input) -> Service:
        return self._services[IService](service_input)
```

However, I think that passing the entire container to a service has a weird smell...

With `Lazy` services, you can pass the `Lazy[IService]` to the `Manager`, and lazily create `Service` instances:

```python
class EntityManager(IEntityManager):
    def __init__(self, lazy_service: Lazy[IService]):
        self._lazy_service = lazy_service
    
    def create_service(service_input):
        return self._lazy_service(service_input)
```


Some fairly basic examples are set out below:

In [1]:
from abc import ABC, abstractmethod

from pyjudo import ServiceContainer, Lazy

In [2]:
# Create some service interfaces and implementations

# Interfaces
class IMyServiceA(ABC):
    test: str
    
    @abstractmethod
    def show_a(self):
        pass

class IMyServiceB(ABC):
    @abstractmethod
    def show_b(self):
        pass

# Implementations
class MyServiceA(IMyServiceA):
    def __init__(self, text: str = "Hello, World!"):
        self.text = text

    def show_a(self):
        print(self.text)

# Note: MyServiceB depends on a Lazy MyServiceA
class MyServiceB(IMyServiceB):
    def __init__(self, service_a: Lazy[IMyServiceA]):
        self.service_a = service_a
    
    def show_b(self):
        print("MyServiceB doing something else")

In [3]:
# Create a service container and add the services

services = ServiceContainer()

services.add_transient(IMyServiceA, MyServiceA)
services.add_transient(IMyServiceB, MyServiceB)

In [4]:
# Resolve an instance of MyServiceB (which depends on MyServiceA)

b1 = services.get(IMyServiceB)

In [5]:
# Use ServiceA component from ServiceB
b1.show_b()
b1.service_a.show_a()

MyServiceB doing something else
Hello, World!


In [6]:
b2 = services.get(IMyServiceB)

In [7]:
# Use ServiceA component from ServiceB
# This time, before accessing any attributes, call it with some overrides

b2.service_a(text="I have been overriden")
b2.service_a.show_a()

I have been overriden


In [8]:
# Create some more service interfaces and implementations
# This time ServiceC has an unresolvable argument without a default value

# Interfaces
class IMyServiceC(ABC):
    test: str
    
    @abstractmethod
    def show_c(self):
        pass

class IMyServiceD(ABC):
    @abstractmethod
    def show_d(self):
        pass

# Implementations
class MyServiceC(IMyServiceC):
    def __init__(self, text: str):
        self.text = text

    def show_c(self):
        print(self.text)

# Note: MyServiceB depends on a Lazy MyServiceA
class MyServiceD(IMyServiceD):
    def __init__(self, service_c: Lazy[IMyServiceC]):
        self.service_c = service_c
    
    def show_d(self):
        print("MyServiceD doing another thing")
        self.service_c.show_c()

In [9]:
services.add_transient(IMyServiceC, MyServiceC)
services.add_transient(IMyServiceD, MyServiceD)

In [10]:
d1 = services.get(IMyServiceD)

In [11]:
# This should cause an exception
# This is because ServiceD accesses ServiceC for the first time 
#  without has not been passed a 'text' argument

from pyjudo.exceptions import ServicesResolutionError

try:
    d1.show_d() # Note: This uses the service_c component
except ServicesResolutionError as e:
    print(e)

MyServiceD doing another thing
Unable to resolve dependency 'text' for '<class '__main__.MyServiceC'>'


In [12]:
# With lazy services, you can pass the arguments when you access the service

d2 = services.get(IMyServiceD)

d2.service_c(text="I now work fine")
d2.show_d()

MyServiceD doing another thing
I now work fine


In [13]:
# Using the Lazy mechanism, you can pass in dependencies to services,
# and only resolve their dependencies when they are accessed. This can be useful
# if you wan to pass in arguments to a service that are not known at the time of registration